# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key

from api_keys import geoapify_key
# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,60.82,48,84,9.37,AU,2024-04-05
1,1,ancud,-41.8697,-73.8203,56.61,75,47,6.26,CL,2024-04-05
2,2,mahibadhoo,3.7833,72.9667,84.74,73,65,6.29,MV,2024-04-05
3,3,petropavlovsk-kamchatsky,53.0452,158.6483,23.59,68,40,2.24,RU,2024-04-05
4,4,blackmans bay,-43.0167,147.3167,50.52,94,2,5.88,AU,2024-04-05


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Configure the map
map_plot_Cities = city_data_df.hvplot.points(
    "Lng",     #  from df columns called Lng
    "Lat",     #  from df columns called Lat
    geo = True,
    tiles = "EsriWorldNavigationCharts",
    frame_width = 950,
    frame_height = 700,
    size = "Humidity",
    scale = 0.55,
    color = "City",
    hover_cols = ["Max Temp", "Humidity", "Cloudiness", "WindSpeed", "Country", "Date"]
)

# Display the map plot
map_plot_Cities

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Max Temp,Cloudiness,Country,Date)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df=city_data_df[(city_data_df['Max Temp'] < 110) &
                             (city_data_df['Max Temp'] >65) &
                             (city_data_df['Wind Speed'] < 30) &
                             (city_data_df['Cloudiness'] < 30) &
                             (city_data_df['Humidity'] < 80)]
ideal_weather_df
# # Drop any rows with null values
ideal_weather_df.dropna(inplace=True)

# # Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,faya-largeau,17.9257,19.1043,89.01,9,3,12.59,TD,2024-04-05
8,8,punta del este,-34.9667,-54.9500,75.31,74,20,10.36,UY,2024-04-05
15,15,rio grande,-32.0350,-52.0986,78.91,53,0,9.46,BR,2024-04-05
22,22,castillos,-34.1667,-53.8333,71.17,58,0,7.92,UY,2024-04-05
24,24,crane,31.3974,-102.3501,81.84,12,4,27.34,US,2024-04-05


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
6,faya-largeau,TD,17.9257,19.1043,9,
8,punta del este,UY,-34.9667,-54.9500,74,
15,rio grande,BR,-32.0350,-52.0986,53,
22,castillos,UY,-34.1667,-53.8333,58,
24,crane,US,31.3974,-102.3501,12,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:

# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key,
    'categories': 'accommodation.hotel',
    'radius': radius,
}

# # Print a message to follow up the hotel search
print("Starting hotel search")

base_url = "https://api.geoapify.com/v2/places"

# loop through checking for params
for index, row in hotel_df.iterrows():
    params['filter'] = f"circle:{row['Lng']},{row['Lat']},{radius}"
    params['bias'] = f"proximity:{row['Lng']},{row['Lat']}"
    params['categories']
 

        # Make and API request using the params dictionaty
    try:
        response = requests.get(base_url, params=params)

        # convert 2 json
        name_address = response.json()
            # if find hotel 
        if name_address["features"]:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
            # if no hotel print this
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
            # handle ERRORS as if no hotel found print this
    except Exception as e:
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    print(f"{row['City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df)

Starting hotel search
faya-largeau - nearest hotel: No hotel found
punta del este - nearest hotel: Hotel Aquarium
rio grande - nearest hotel: Hotel Vila Moura Executivo
castillos - nearest hotel: A mi gente
crane - nearest hotel: No hotel found
callao - nearest hotel: Casa Ronald
nellore - nearest hotel: Grand MINERVA
puerto real - nearest hotel: Hostal Bahía de Cádiz
hadibu - nearest hotel: No hotel found
coquimbo - nearest hotel: Hotel Iberia
saipan - nearest hotel: Chalan Kanoa Beach Hotel
dawesville - nearest hotel: Baymont by Wyndham Thomasville
carnarvon - nearest hotel: No hotel found
nasiriyah - nearest hotel: فندق حمورابي
progreso - nearest hotel: Centro Vacacional Obrero CTM
gonzalez - nearest hotel: Courtyard Pensacola West
cape canaveral - nearest hotel: Holiday Inn Club Vacations Cape Canaveral Beach Resort
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
kolonia town - nearest hotel: Cliff Rainbow Hotel
khuzdar - nearest hotel: Al-Mustafa hotel
ingeniero jac

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
hotel_df


In [ ]:
%%capture --no-display


# Configure the map
map_plot_Cities_part_DUO = hotel_df.hvplot.points(
    "Lng",     #  from df columns called Lng
    "Lat",     #  from df columns called Lat
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 950,
    frame_height = 700,
    size = "Humidity",
    scale = 0.55,
    color = "City",
    hover_cols = ["City", "Hotel Name", "Country"]
)

# Display the map plot
map_plot_Cities_part_DUO